# Historical Returns Distribution fitting

## Normal or not

Before using the Volatility infered from the log-returns normality assumption, it is worth looking into the validity of that assumption.



## Normal Distribution


## Laplace Distribution

## Lookback at the sigma levels